# Header

In [ ]:
import numpy as np
from ipywidgets import widgets
from Functions.Properties_Functions import * 

# Equipment Functions

In [ ]:
def Cal_Seperator(z,F,P,T):
    Tr = T/Tc
    alpha = (1 + k*(1 - np.sqrt(Tr)))**2
    a_alpha = a*alpha
    a_alphaij = np.sqrt(np.array([a_alpha]).T.dot(np.array([a_alpha])))*(1-binary_coeff)

    Psat = Antoine(T, AA, AB, AC)
    K = Psat/P

    Fr = 0.5
    f_Fr = sigma_Fr(Fr,z,K).sum()
    fp_Fr = sigmap_Fr(Fr,z,K).sum()

    while np.abs(f_Fr) > 1.0e-10:
        Fr = Fr - f_Fr/fp_Fr
        f_Fr = sigma_Fr(Fr,z,K).sum()
        fp_Fr = sigmap_Fr(Fr,z,K).sum()
    Fr0 = Fr
    x = z/(1 - Fr*(1-K))
    y = x*K

    a_alphal = np.sum(np.array([x]).T.dot(np.array([x]))*a_alphaij)
    bl = np.sum(x*b)
    Al = a_alphal*P/((R*T)**2)
    Bl = bl*P/(R*T)

    a_alphag = np.sum(np.array([y]).T.dot(np.array([y]))*a_alphaij)
    bg = np.sum(y*b)
    Ag = a_alphag*P/((R*T)**2)
    Bg = bg*P/(R*T)
    
    Zl = cardan(Al,Bl)
    if np.size(Zl) > 2:
      Zl = min(Zl)
    Zg = cardan(Ag,Bg)
    if np.size(Zg) > 2:
      Zg = max(Zg)
    vl = Zl * R * T / P
    vg = Zg * R * T / P

    xa_alpha = np.array([x]).dot(a_alphaij)
    xa_alpha = xa_alpha[0]
    ya_alpha = np.array([y]).dot(a_alphaij)
    ya_alpha = ya_alpha[0]
    fl_log = b/bl*(Zl-1) - np.log((vl-bl)*P/(R*T)) + \
              a_alphal/(2*np.sqrt(2)*bl*R*T)*(b/bl-2/a_alphal*xa_alpha)*np.log((vl+(1+2**0.5)*bl)/(vl+(1-2**0.5)*bl))
    fg_log = b/bg*(Zg-1) - np.log((vg-bg)*P/(R*T)) + \
              a_alphag/(2*np.sqrt(2)*bg*R*T)*(b/bg-2/a_alphag*ya_alpha)*np.log((vg+(1+2**0.5)*bg)/(vg+(1-2**0.5)*bg))

    fl = np.exp(fl_log)
    fg = np.exp(fg_log)

    K = fl/fg
    Fr = 0.5
    f_Fr = sigma_Fr(Fr,z,K).sum()
    fp_Fr = sigmap_Fr(Fr,z,K).sum()

    while np.abs(f_Fr) > 1.0e-10:
      Fr = Fr - f_Fr/fp_Fr
      f_Fr = sigma_Fr(Fr,z,K).sum()
      fp_Fr = sigmap_Fr(Fr,z,K).sum()
    x = z/(1 - Fr*(1-K))
    y = x*K
    
    while abs(Fr - Fr0) > 1.0e-10:

      Fr0 = Fr

      a_alphal = np.sum(np.array([x]).T.dot(np.array([x]))*a_alphaij)
      bl = np.sum(x*b)
      Al = a_alphal*P/((R*T)**2)
      Bl = bl*P/(R*T)

      a_alphag = np.sum(np.array([y]).T.dot(np.array([y]))*a_alphaij)
      bg = np.sum(y*b)
      Ag = a_alphag*P/((R*T)**2)
      Bg = bg*P/(R*T)

      Zl = cardan(Al,Bl)
      if np.size(Zl) > 2:
        Zl = min(Zl)
      Zg = cardan(Ag,Bg)
      if np.size(Zg) > 2:
        Zg = max(Zg)
      vl = Zl * R * T / P
      vg = Zg * R * T / P

      xa_alpha = np.array([x]).dot(a_alphaij)
      xa_alpha = xa_alpha[0]
      ya_alpha = np.array([y]).dot(a_alphaij)
      ya_alpha = ya_alpha[0]
      fl_log = b/bl*(Zl-1) - np.log((vl-bl)*P/(R*T)) + \
                a_alphal/(2*np.sqrt(2)*bl*R*T)*(b/bl-2/a_alphal*xa_alpha)*np.log((vl+(1+2**0.5)*bl)/(vl+(1-2**0.5)*bl))
      fg_log = b/bg*(Zg-1) - np.log((vg-bg)*P/(R*T)) + \
                a_alphag/(2*np.sqrt(2)*bg*R*T)*(b/bg-2/a_alphag*ya_alpha)*np.log((vg+(1+2**0.5)*bg)/(vg+(1-2**0.5)*bg))

      fl = np.exp(fl_log)
      fg = np.exp(fg_log)

      K = fl/fg
      Fr = 0.5
      f_Fr = sigma_Fr(Fr,z,K).sum()
      fp_Fr = sigmap_Fr(Fr,z,K).sum()

      while np.abs(f_Fr) > 1.0e-10:
        Fr = Fr - f_Fr/fp_Fr
        f_Fr = sigma_Fr(Fr,z,K).sum()
        fp_Fr = sigmap_Fr(Fr,z,K).sum()
      x = z/(1 - Fr*(1-K))
      y = x*K



    Fv = F*Fr
    Fl = F - Fv
    
    OutString_Fraction = "NULL"
    OutString_xFraction = "NULL"
    OutString_yFraction = "NULL"

    if Fv < 0 or Fl <0 :
      print("Error: It dosen't converge")
      OutString_Fraction = "Error: It dosen't converge"
    else:
      print('F_V:',np.round(Fv,2),', F_L:',np.round(Fl,2))
      print('x0:',np.round(x[0],4),', x1:',np.round(x[1],4),', x2:',np.round(x[2],4),', x3:',np.round(x[3],4),', x4:',np.round(x[4],4))
      print('y0:',np.round(y[0],4),', y1:',np.round(y[1],4),', y2:',np.round(y[2],4),', y3:',np.round(y[3],4),', y4:',np.round(y[4],4))
      OutString_Fraction = "Vapor Amount : " + str(np.round(Fv,2)) + "  /  Liquid Amount : " + str(np.round(Fl,2)) + "\n"
      OutString_xFraction = 'x_N2:' + str(np.round(x[0],4)) + ', x_C1:' + str(np.round(x[1],4)) + ', x_C2:' + str(np.round(x[2],4)) + ', x_C3:'+ str(np.round(x[3],4)) + ', x:C4:' + str(np.round(x[4],4)) + "\n"
      OutString_yFraction = 'y_N2:' + str(np.round(y[0],4)) + ', y_C1:' + str(np.round(y[1],4)) + ', y_C2:' + str(np.round(y[2],4)) + ', y_C3:'+ str(np.round(y[3],4)) + ', y_C4:' + str(np.round(y[4],4)) + "\n"
    
    return [OutString_Fraction,OutString_xFraction,OutString_yFraction]

In [ ]:
return_text = Cal_Seperator(np.array([0.3, 0.1, 0.0, 0.2, 0.4]),100,1.0e5,-24.6189 + 273.15)
print(return_text[0],return_text[1],return_text[2])

F_V: 90.2 , F_L: 9.8
x0: 0.0006 , x1: 0.001 , x2: 0.0 , x3: 0.1045 , x4: 0.8939
y0: 0.3325 , y1: 0.1108 , y2: 0.0 , y3: 0.2104 , y4: 0.3463
Vapor Amount : 90.2  /  Liquid Amount : 9.8
 x_N2:0.0006, x_C1:0.001, x_C2:0.0, x_C3:0.1045, x:C4:0.8939
 y_N2:0.3325, y_C1:0.1108, y_C2:0.0, y_C3:0.2104, y_C4:0.3463



# TEST 

In [ ]:
# N2, C1, C2, C3, C4
z = np.array([0.0, 0.0, 0.0, 0.5, 0.5])
P1 = 1.0e5
T1 = -24.6189
F = 100
result_string = Cal_Seperator(z,F,P1,T1+273.15)
print(result_string)

F_V: 32.18 , F_L: 67.82
x0: 0.0 , x1: 0.0 , x2: 0.0 , x3: 0.378 , x4: 0.622
y0: 0.0 , y1: 0.0 , y2: 0.0 , y3: 0.757 , y4: 0.243
['Vapor Amount : 32.18  /  Liquid Amount : 67.82\n', 'x_N2:0.0, x_C1:0.0, x_C2:0.0, x_C3:0.378, x:C4:0.622\n', 'y_N2:0.0, y_C1:0.0, y_C2:0.0, y_C3:0.757, y_C4:0.243\n']


# Widget

In [ ]:
Label1 = widgets.Label("Composition")
wComp1 = widgets.BoundedFloatText(
    value=0.0,
    min=0,
    max=1.0,
    step=0.1,
    description='N2:',
    disabled=False
)
wComp2 = widgets.BoundedFloatText(
    value=0.0,
    min=0,
    max=1.0,
    step=0.1,
    description='C1:',
    disabled=False
)
wComp3 = widgets.BoundedFloatText(
    value=0.0,
    min=0,
    max=1.0,
    step=0.1,
    description='C2:',
    disabled=False
)
wComp4 = widgets.BoundedFloatText(
    value=0.5,
    min=0,
    max=1.0,
    step=0.1,
    description='C3:',
    disabled=False
)
wComp5 = widgets.BoundedFloatText(
    value=0.5,
    min=0,
    max=1.0,
    step=0.1,
    description='C4:',
    disabled=False
)
wCompSum = widgets.BoundedFloatText(
    value=wComp1.value+wComp2.value+wComp3.value+wComp4.value+wComp5.value,
    min=0,
    max=1.0,
    step=0.1,
    description='SUM:',
    disabled=False
)

Label2 = widgets.Label("Input")

wInputPressure = widgets.BoundedFloatText(
    value=1,
    min=0,
    max=300.0,
    step=0.1,
    description='P1 (bar) :',
    disabled=False
)
wInputTemperature = widgets.BoundedFloatText(
    value=-24.6,
    min=-500,
    max=500.0,
    step=0.1,
    description='T1 (C) :',
    disabled=False
)

wInputFlow = widgets.BoundedFloatText(
    value=100,
    min=-100000,
    max=100000,
    step=0.1,
    description='F :',
    disabled=False
)

wOutputPressure = widgets.BoundedFloatText(
    value=2,
    min=0,
    max=300.0,
    step=0.1,
    description='P2 (bar) :',
    disabled=False
)

Label3 = widgets.Label("< ### Result ### >")

wOutputTemperature = widgets.BoundedFloatText(
    value=0,
    min=-500,
    max=500.0,
    step=0.1,
    description='T2 (C) :',
    disabled=False
)

wOutputEnergy = widgets.BoundedFloatText(
    value=0,
    min=-1000000,
    max=1000000,
    step=0.1,
    description='W(Q) (KJ/Kmol) :',
    disabled=False
)

StringOut_Fraction = widgets.Label()
StringOut_xFraction = widgets.Label()
StringOUt_yFraction = widgets.Label()

btn = widgets.Button(description = "Calculate")
lbl3 = widgets.Label()

def Calculate_Seperator(a):
    z = np.array([wComp1.value,wComp2.value,wComp3.value,wComp4.value,wComp5.value])
    P1 = wInputPressure.value * 1e5
    T1 = wInputTemperature.value + 273.15
    F = wInputFlow.value
    resut_String = Cal_Seperator(z,F,P1,T1)
    
    StringOut_Fraction.value = resut_String[0]
    StringOut_xFraction.value = resut_String[1]
    StringOUt_yFraction.value = resut_String[2]

# Seperator

In [ ]:
display(widgets.Label("### Seperator Calculation ###"))
display(Label1,wComp1,wComp2,wComp3,wComp4,wComp5)
display(Label2,wInputPressure,wInputTemperature,wInputFlow)
display(btn)
display(StringOut_Fraction,StringOut_xFraction,StringOUt_yFraction)
btn.on_click(Calculate_Seperator)

Label(value='### Seperator Calculation ###')

Label(value='Composition')

BoundedFloatText(value=0.0, description='N2:', max=1.0, step=0.1)

BoundedFloatText(value=0.0, description='C1:', max=1.0, step=0.1)

BoundedFloatText(value=0.0, description='C2:', max=1.0, step=0.1)

...

BoundedFloatText(value=100.0, description='F :', max=100000.0, min=-100000.0, step=0.1)

Button(description='Calculate', style=ButtonStyle())

Label(value='')

Label(value='')

Label(value='')